In [60]:
import pandas as pd
from pathlib import Path
import re

In [61]:
file_path = Path('.').resolve() / 'data' / 'recipt_content.xlsx'
recipe_content = pd.read_excel(file_path, index_col=0).rename({0:'content'},axis = 1).dropna().reset_index(drop=True)
recipe_content.head(15).iloc[10].content
print(recipe_content.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   content     681 non-null    object 
 1   is_product  681 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.8+ KB
None


In [62]:
def extract_price(text_content:str):
    t =  text_content.replace(':','0')
    pattern = r'[^.,-]\.?(\s?\d+[,.]+?\d\d\s?[0OABC846]\s)'
    if re.search(pattern,t) and len(t) > 15:
        price = re.findall(pattern, t)
        return price[0]
    return None

recipe_content['price'] = recipe_content['content'].apply(extract_price)

In [63]:
only_products = recipe_content.loc[recipe_content['is_product'] == 1]

In [64]:
print(only_products.loc[only_products['price'].isnull()])
only_products

Empty DataFrame
Columns: [content, is_product, price]
Index: []


,content,is_product,price
10,VICHY Dercos szapp/łup wrazliwy. 17841A 1op 44...,1.0,"44,99 0"
32,"BLOK BIUROWY A4 100K TOP COLOR 2.0 1SZT x5,20 ...",1.0,"5,20A"
33,"ZESZYT AS 60K KRAFT 1SZT X3,00 3,00A",1.0,"3,00A"
34,"ZESZYT A5 32K PP TOP 1SZT X2,80 2,80A",1.0,"2,80A"
35,"KOPERTY DO ZAPROSZEN METALICZNE 2SZT 1,00 2,00A",1.0,"2,00A"
...,...,...,...
615,"TYMBARK SOK MULTIVEJ 1 x2,49 2,490",1.0,"2,490"
617,"EKA SCHMITT PŁYTK\AX 1 X24,99 24.99A",1.0,24.99A
619,"KUPIEC WAFLE RYZOVEJ 1 x2,19 2,190",1.0,"2,190"
641,"2004714399999 WODA TOALETO 1*59,90= 59,90 A",1.0,"59,90 A"


In [65]:
only_recognised = recipe_content.loc[recipe_content['price'].notnull()]

In [66]:
print(only_recognised.value_counts('is_product'))
print(only_recognised.loc[only_recognised['is_product'] == 0])
only_recognised
# can be fixed by specifying demanded length

is_product
1.0    182
dtype: int64
Empty DataFrame
Columns: [content, is_product, price]
Index: []


,content,is_product,price
10,VICHY Dercos szapp/łup wrazliwy. 17841A 1op 44...,1.0,"44,99 0"
32,"BLOK BIUROWY A4 100K TOP COLOR 2.0 1SZT x5,20 ...",1.0,"5,20A"
33,"ZESZYT AS 60K KRAFT 1SZT X3,00 3,00A",1.0,"3,00A"
34,"ZESZYT A5 32K PP TOP 1SZT X2,80 2,80A",1.0,"2,80A"
35,"KOPERTY DO ZAPROSZEN METALICZNE 2SZT 1,00 2,00A",1.0,"2,00A"
...,...,...,...
615,"TYMBARK SOK MULTIVEJ 1 x2,49 2,490",1.0,"2,490"
617,"EKA SCHMITT PŁYTK\AX 1 X24,99 24.99A",1.0,24.99A
619,"KUPIEC WAFLE RYZOVEJ 1 x2,19 2,190",1.0,"2,190"
641,"2004714399999 WODA TOALETO 1*59,90= 59,90 A",1.0,"59,90 A"


In [67]:
pd.concat([only_recognised,only_products]).drop_duplicates()

,content,is_product,price
10,VICHY Dercos szapp/łup wrazliwy. 17841A 1op 44...,1.0,"44,99 0"
32,"BLOK BIUROWY A4 100K TOP COLOR 2.0 1SZT x5,20 ...",1.0,"5,20A"
33,"ZESZYT AS 60K KRAFT 1SZT X3,00 3,00A",1.0,"3,00A"
34,"ZESZYT A5 32K PP TOP 1SZT X2,80 2,80A",1.0,"2,80A"
35,"KOPERTY DO ZAPROSZEN METALICZNE 2SZT 1,00 2,00A",1.0,"2,00A"
...,...,...,...
615,"TYMBARK SOK MULTIVEJ 1 x2,49 2,490",1.0,"2,490"
617,"EKA SCHMITT PŁYTK\AX 1 X24,99 24.99A",1.0,24.99A
619,"KUPIEC WAFLE RYZOVEJ 1 x2,19 2,190",1.0,"2,190"
641,"2004714399999 WODA TOALETO 1*59,90= 59,90 A",1.0,"59,90 A"
